In [1]:
import torch
from torch.autograd import Variable ## torch中自动计算梯度模块
import torch.nn as nn # 神经网络模块
import torch.utils.data as data
import torch.nn.functional as F  #神经网络模块中的常用功能 
import torch.multiprocessing as mp
from torch import optim
from torch.optim import lr_scheduler
from torch.utils.data import DataLoader,Dataset

import torchvision
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from torchvision.utils import make_grid
from torchvision import datasets, models
from torchvision.models import *

from __future__ import print_function, division
import matplotlib.pyplot as plt
import numpy as np
import random
import cv2
import time
import os
import copy
from tqdm import tqdm
import pandas as pd
#from skimage import io, transform
from PIL import Image

%matplotlib inline
plt.ion()   # interactive mode
%config InlineBackend.figure_format = 'retina'

#显示中文字体设置
plt.rcParams["font.sans-serif"] = ["Droid Sans Fallback"]
plt.rcParams['axes.unicode_minus'] = False #为了正常显示是"-"减号

In [2]:
classes = ['collar_design_labels', 'neck_design_labels'] 
label_count = {'coat_length_labels':8,
               'collar_design_labels':5, 
               'lapel_design_labels':5,
               'neck_design_labels':5,
               'neckline_design_labels':10,
               'pant_length_labels':6, 
               'skirt_length_labels':6, 
               'sleeve_length_labels':9}
os.environ["CUDA_VISIBLE_DEVICES"] = "2,3"
#若gpu可用则返回True
use_gpu = torch.cuda.is_available()

#保存文件后缀,即月份和日为版本尾号
#version = ""
version = '0404'
image_width = 224
epochs_num = 25
learning_rate = 0.001
split_ratio=0.2
batch_size = 16

import Augmentor
import torchvision
p = Augmentor.Pipeline()
p.rotate(probability=0.7, max_left_rotation=5, max_right_rotation=5)
p.zoom(probability=0.5, min_factor=1.1, max_factor=1.5)


In [3]:
#定义数据预处理
fai_data_transforms = {
                    'train': transforms.Compose([
                        transforms.Resize(224, interpolation=2),
                        p.torch_transform(),
                        transforms.RandomHorizontalFlip(),
                        transforms.ColorJitter(brightness=0.3, contrast=0.2, saturation=0.2, hue=0.2),
                        transforms.RandomRotation(20, resample=False, expand=False, center=None),
                        transforms.RandomResizedCrop(image_width, scale=(0.95, 1.0), ratio=(0.75, 1.3333333333333333), interpolation=2),
                        
                        transforms.ToTensor(),
                        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])]),
                    'val': transforms.Compose([
                        #transforms.Resize(256),
                        #transforms.CenterCrop(256,224),
                        transforms.Resize(224, interpolation=2),
                        transforms.ToTensor(),
                        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])}
fai_test_data_transforms=transforms.Compose([
                        transforms.Resize(224),
                        p.torch_transform(),
                        #transforms.CenterCrop(256,224),
                        transforms.Resize(224, interpolation=2),
                        transforms.ToTensor(),
                        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])
#定义数据集
def default_loader(path):
    return Image.open(path).convert('RGB')

class FaiTrainDataset(Dataset):
    """FaiTrainDataset dataset."""

    def __init__(self, csv_path_and_file, train_images_root_dir,split_ratio=0.2,transform=None,target_transform=None,loader=default_loader):
        """
        Args:
            csv_file (string): Path to the csv file with annotations.
            root_dir (string): Directory with all the images.
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """
        self.df = pd.read_csv(csv_path_and_file)
        #self.df_load = df.sample(frac=1).reset_index(drop=True)
        self.df_load = self.df.copy()
        #df.iloc[np.random.permutation(len(df))]
        #self.x, self.y = self.df_load.ix[:,0],self.df_load.ix[:,2:]
        #self.x_train,self.x_test,self.y_train,self.y_test = train_test_split(self.x,self.y,test_size=split_ratio,random_state=42)
        self.split_ratio = split_ratio
        self.cut_idx = int(len(self.df_load)-round(len(self.df_load)*self.split_ratio))#int(round(0.1 * self.df_load.shape[0]))
        #self.cut_idx = int(round(len(self.df_load)*self.split_ratio))
        #self.b = int(len(self.df_load)*0.2)
   
        self.df_train= self.df_load.iloc[:self.cut_idx]
    
        self.df_train_load = self.df_train.copy()
        self.df_train_load.columns = ['image_id', 'class', 'label']
        self.df_train_load.reset_index(drop=True)
        
        self.train_images = self.df_train_load['image_id'].tolist()
        self.train_labels = self.df_train_load['label'].tolist()
        n=len(self.df_train_load)
        self.y = np.zeros((n, label_count[self.df_train_load['class'][0]]), dtype=np.uint8)
        for i in range(n):
            tmp_label=self.train_labels[i]
            self.y[i][tmp_label.find('y')] = 1
        
        self.train_data = list(zip(self.train_images,self.y))
        print(self.train_data[0])
        
        self.train_images_root_dir = train_images_root_dir
        self.transform = transform
        self.target_transform = target_transform
        self.loader = loader
       
    def __len__(self):
        
        return len(self.df_train_load)

    def __getitem__(self, index): #最终返回的是Tensor
        
        image_path, label = self.train_data[index]
        img_name = os.path.join(self.train_images_root_dir,image_path)
        #image = io.imread(img_name) #用的skimage.io,读入为uint8，RGB，HWc图像
        img = self.loader(img_name)
            
        if self.transform is not None:
            img = self.transform(img) #处理后为RGB CHW 0-1.0数据
        return img,label
class FaiValDataset(Dataset):
    """FaiTrainDataset dataset."""

    def __init__(self, csv_path_and_file, train_images_root_dir,split_ratio=0.2,transform=None,target_transform=None,loader=default_loader):
        """
        Args:
            csv_file (string): Path to the csv file with annotations.
            root_dir (string): Directory with all the images.
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """
        self.df = pd.read_csv(csv_path_and_file)
        #self.df_load = df.sample(frac=1).reset_index(drop=True)
        self.df_load = self.df.copy()
        #df.iloc[np.random.permutation(len(df))]
        #self.x, self.y = self.df_load.ix[:,0],self.df_load.ix[:,2:]
        #self.x_train,self.x_test,self.y_train,self.y_test = train_test_split(self.x,self.y,test_size=split_ratio,random_state=42)
        self.split_ratio = split_ratio
        self.cut_idx = int(len(self.df_load)-round(len(self.df_load)*self.split_ratio))#int(round(0.1 * self.df_load.shape[0]))
        #self.cut_idx = int(round(len(self.df_load)*self.split_ratio))
        #self.b = int(len(self.df_load)*0.2)
   
        self.df_val= self.df_load.iloc[self.cut_idx:]
    
        self.df_val_load = self.df_val.copy()
        self.df_val_load.columns = ['image_id', 'class', 'label']
        self.df_val_load.reset_index(inplace= True,drop=True)
        
        self.val_images = self.df_val_load['image_id'].tolist()
        self.val_labels = self.df_val_load['label'].tolist()
        n=len(self.df_val_load)
        #print(self.df_val_load['class'])
        self.yy = np.zeros((n, label_count[self.df_val_load['class'][0]]), dtype=np.uint8)
       
        for i in range(n):
            tmp_label=self.val_labels[i]
            self.yy[i][tmp_label.find('y')] = 1
        
        self.val_data = list(zip(self.val_images,self.yy))
        print(self.val_data[0])
        
        self.train_images_root_dir = train_images_root_dir
        self.transform = transform
        self.target_transform = target_transform
        self.loader = loader
       
    def __len__(self):
        
        return len(self.df_val_load)

    def __getitem__(self, index): #最终返回的是Tensor
        
        image_path, label = self.val_data[index]
        img_name = os.path.join(self.train_images_root_dir,image_path)
        #image = io.imread(img_name) #用的skimage.io,读入为uint8，RGB，HWc图像
        img = self.loader(img_name)
            
        if self.transform is not None:
            img = self.transform(img) #处理后为RGB CHW 0-1.0数据
        return img,label

#定义测试数据集
class FaiTestDataset(Dataset):
    """FaiTestDataset dataset."""

    def __init__(self, csv_path_and_file, test_images_root_dir,split_ratio=0.0,transform=None,target_transform=None,loader=default_loader):
        """
        Args:
            csv_file (string): Path to the csv file with annotations.
            root_dir (string): Directory with all the images.
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """
        self.df = pd.read_csv(csv_path_and_file)
        #self.df_load = df.sample(frac=1).reset_index(drop=True)
        self.df_load = self.df.copy()
        #df.iloc[np.random.permutation(len(df))]
        #self.x, self.y = self.df_load.ix[:,0],self.df_load.ix[:,2:]
        #self.x_train,self.x_test,self.y_train,self.y_test = train_test_split(self.x,self.y,test_size=split_ratio,random_state=42)
        self.split_ratio = split_ratio
        self.cut_idx = int(len(self.df_load)-round(len(self.df_load)*self.split_ratio))#int(round(0.1 * self.df_load.shape[0]))
        #self.cut_idx = int(round(len(self.df_load)*self.split_ratio))
        #self.b = int(len(self.df_load)*0.2)
   
        self.df_test= self.df_load.iloc[:self.cut_idx]
    
        self.df_test_load = self.df_test.copy()
        self.df_test_load.columns = ['image_id', 'class', 'label']
        self.df_test_load.reset_index(inplace= True,drop=True)
        
        self.test_images = self.df_test_load['image_id'].tolist()
        self.test_labels = self.df_test_load['label'].tolist()
        n=len(self.df_test_load)
        #print(self.df_test_load['class'])
        self.yyy = np.zeros((n, label_count[self.df_test_load['class'][0]]), dtype=np.uint8)
       
        for i in range(n):
            tmp_label=self.test_labels[i]
            self.yyy[i][tmp_label.find('?')] = 1
        
        self.test_data = list(zip(self.test_images,self.yyy))
        print(self.test_data[0])
        
        self.test_images_root_dir = test_images_root_dir
        self.transform = transform
        self.target_transform = target_transform
        self.loader = loader
       
    def __len__(self):
        
        return len(self.df_test_load)

    def __getitem__(self, index): #最终返回的是Tensor
        
        image_path, label = self.test_data[index]
        img_name = os.path.join(self.test_images_root_dir,image_path)
        #image = io.imread(img_name) #用的skimage.io,读入为uint8，RGB，HWc图像
        img = self.loader(img_name)
            
        if self.transform is not None:
            img = self.transform(img) #处理后为RGB CHW 0-1.0数据
        return img,label

    
    
fai_train_dataset = FaiTrainDataset(csv_path_and_file='../train/Annotations/{0}.csv'.format(classes[0]),
                                   train_images_root_dir='../train/',
                                   split_ratio=split_ratio,
                                   transform = fai_data_transforms['train'])
fai_val_dataset = FaiValDataset(csv_path_and_file='../train/Annotations/{0}.csv'.format(classes[0]),
                                   train_images_root_dir='../train/',
                                   split_ratio=split_ratio,
                                   transform = fai_data_transforms['val'])

fai_test_dataset = FaiTestDataset(csv_path_and_file='../test/Tests/{0}.csv'.format(classes[0]),
                                  test_images_root_dir='../test/',
                                  transform = fai_test_data_transforms)


train_dataloader = torch.utils.data.DataLoader(fai_train_dataset, batch_size=2,shuffle=True, num_workers=4)
val_dataloader = torch.utils.data.DataLoader(fai_val_dataset, batch_size=2,shuffle=True, num_workers=1)
test_dataloader = torch.utils.data.DataLoader(fai_test_dataset, batch_size=2,shuffle=True, num_workers=4)
#test_dataloader = torch.utils.data.DataLoader(fai_test_dataset, batch_size=4,shuffle=False, num_workers=4)
print(len(fai_test_dataset))

print(next(iter(test_dataloader)))

print(len(fai_train_dataset))
print(len(fai_val_dataset))
#next(iter(train_dataloader))
print(next(iter(train_dataloader)))
print(next(iter(val_dataloader)))
samples,labels = next(iter(train_dataloader))
print(samples.size(),labels.size())


('Images/collar_design_labels/bd0981f231180d2b001d4a37e9834630.jpg', array([0, 1, 0, 0, 0], dtype=uint8))
('Images/collar_design_labels/eae3054c14ff6463205ecaadee005251.jpg', array([0, 0, 0, 0, 1], dtype=uint8))
('Images/collar_design_labels/677e1183282769a3fe8ac5f1f0154bbd.jpg', array([1, 0, 0, 0, 0], dtype=uint8))
1081
立波 [<PIL.Image.Image image mode=RGB size=224x224 at 0x7F9E751896A0>]
立波 [<PIL.Image.Image image mode=RGB size=224x224 at 0x7F9E75189668>]
立波 [<PIL.Image.Image image mode=RGB size=224x224 at 0x7F9E75189828>]
立波 [<PIL.Image.Image image mode=RGB size=224x224 at 0x7F9E751897B8>]
立波 [<PIL.Image.Image image mode=RGB size=224x224 at 0x7F9E75189630>]
立波 [<PIL.Image.Image image mode=RGB size=224x224 at 0x7F9E75189710>]
立波 [<PIL.Image.Image image mode=RGB size=224x224 at 0x7F9E75189B00>]
立波 [<PIL.Image.Image image mode=RGB size=224x224 at 0x7F9E75189B38>]
立波 [<PIL.Image.Image image mode=RGB size=224x224 at 0x7F9E75189748>]
立波 [<PIL.Image.Image image mode=RGB size=224x224 at 0x7F

立波 [<PIL.Image.Image image mode=RGB size=224x224 at 0x7F9EDDFE8080>]
[
( 0 , 0 ,.,.) = 
 -0.8164 -0.7822 -0.7479  ...  -0.5767 -0.5767 -0.5596
 -0.7822 -0.5596  0.8789  ...  -0.5596 -0.5767 -0.5596
 -0.7650 -0.2684  1.7009  ...  -0.5596 -0.5767 -0.5596
           ...             ⋱             ...          
 -0.2342 -0.2171 -0.2342  ...  -0.7308 -0.7479 -0.7479
 -0.2513 -0.2342 -0.2513  ...  -0.7308 -0.7479 -0.7479
 -0.2342 -0.2342 -0.2513  ...  -0.7479 -0.7137 -0.7308

( 0 , 1 ,.,.) = 
 -0.5651 -0.5826 -0.5826  ...  -0.3901 -0.3901 -0.3725
 -0.5826 -0.4076  1.0280  ...  -0.3725 -0.3901 -0.3725
 -0.5826 -0.1099  1.8683  ...  -0.3725 -0.3901 -0.3725
           ...             ⋱             ...          
  0.0651  0.0826  0.0651  ...  -0.5301 -0.5126 -0.5126
  0.0476  0.0651  0.0476  ...  -0.5301 -0.5126 -0.5126
  0.0651  0.0651  0.0476  ...  -0.5301 -0.4776 -0.4951

( 0 , 2 ,.,.) = 
 -0.2010 -0.2358 -0.2707  ...   0.0082 -0.0092  0.0082
 -0.2358 -0.0964  1.3328  ...   0.0431  0.0256  0.0